# Assign ISO to GTSM / CIAM points
- assign based on country Voronoi
- countries are defined by impact regions

INPUTS
- `sset.PATH_CIAM_SITES`
- `sset.PATH_GADM_ADM0_VORONOI`

OUTPUTS
- `sset.PATH_CIAM_SITES_WITHISO`

In [1]:
import geopandas as gpd
from sliiders import settings as sset

import warnings

warnings.filterwarnings("ignore", message=".*initial implementation of Parquet.*")

In [2]:
countries = gpd.read_parquet(sset.PATH_GADM_ADM0_VORONOI).set_crs(epsg=4326)

In [3]:
stations_wo_country = gpd.read_file(sset.PATH_CIAM_SITES)

In [4]:
stations = gpd.sjoin(stations_wo_country, countries, how="left", predicate="within")

stations.shape[0]

9592

#### Check that match is one-to-one

Check there is no more than one match for each station

In [5]:
assert stations.shape[0] == stations_wo_country.shape[0]

Check there is at least one match for each station

In [6]:
assert stations["ISO"].isnull().sum() == 0

#### Clean and save

In [7]:
stations = stations.drop(columns=["index_right"])

In [8]:
sset.PATH_CIAM_SITES_WITHISO.parent.mkdir(exist_ok=True)

stations[["station_id", "lat", "lon", "geometry", "ISO"]].to_parquet(
    sset.PATH_CIAM_SITES_WITHISO, index=False
)